In [55]:
from google.colab import files

uploaded = files.upload()

Saving Dataset .csv to Dataset  (1).csv


In [92]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from geopy.distance import geodesic
from scipy.spatial.distance import euclidean

# Load the dataset
res_data = pd.read_csv('Dataset .csv')
res_data.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [93]:
import warnings
warnings.filterwarnings("ignore")

**1) EDA**

In [94]:
res_data.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

**Data Cleaning**

In [95]:
res_data=res_data.drop(columns=['Country Code','City','Address','Locality','Locality Verbose','Currency','Has Table booking','Has Online delivery','Is delivering now','Switch to order menu','Rating color','Rating text'])
res_data

,Restaurant ID,Restaurant Name,Longitude,Latitude,Cuisines,Average Cost for two,Price range,Aggregate rating,Votes
0,6317637,Le Petit Souffle,121.027535,14.565443,"French, Japanese, Desserts",1100,3,4.8,314
1,6304287,Izakaya Kikufuji,121.014101,14.553708,Japanese,1200,3,4.5,591
2,6300002,Heat - Edsa Shangri-La,121.056831,14.581404,"Seafood, Asian, Filipino, Indian",4000,4,4.4,270
3,6318506,Ooma,121.056475,14.585318,"Japanese, Sushi",1500,4,4.9,365
4,6314302,Sambo Kojin,121.057508,14.584450,"Japanese, Korean",1500,4,4.8,229
...,...,...,...,...,...,...,...,...,...
9546,5915730,Naml۱ Gurme,28.977392,41.022793,Turkish,80,3,4.1,788
9547,5908749,Ceviz A��ac۱,29.041297,41.009847,"World Cuisine, Patisserie, Cafe",105,3,4.2,1034
9548,5915807,Huqqa,29.034640,41.055817,"Italian, World Cuisine",170,4,3.7,661
9549,5916112,A���k Kahve,29.036019,41.057979,Restaurant Cafe,120,4,4.0,901


In [96]:
#handling missing values
res_data.isnull().sum()

Restaurant ID           0
Restaurant Name         0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Price range             0
Aggregate rating        0
Votes                   0
dtype: int64

In [97]:
res_data = res_data.dropna(subset=['Cuisines'])

**2) Data Preprocessing**

In [98]:
#saving the original data by duplicating
res_data['avg_cost_for_two']=res_data['Average Cost for two'].copy()
res_data['price_range']=res_data['Price range'].copy()
res_data['aggregate_rating']=res_data['Aggregate rating'].copy()
res_data['votes']=res_data['Votes'].copy()

# Normalize numerical columns
scaler = StandardScaler()
res_data[['Average Cost for two', 'Price range', 'Aggregate rating', 'Votes']] = scaler.fit_transform(res_data[['Average Cost for two', 'Price range', 'Aggregate rating', 'Votes']])
res_data

,Restaurant ID,Restaurant Name,Longitude,Latitude,Cuisines,Average Cost for two,Price range,Aggregate rating,Votes,avg_cost_for_two,price_range,aggregate_rating,votes
0,6317637,Le Petit Souffle,121.027535,14.565443,"French, Japanese, Desserts",-0.006221,1.319726,1.407683,0.365493,1100,3,4.8,314
1,6304287,Izakaya Kikufuji,121.014101,14.553708,Japanese,-0.000020,1.319726,1.209860,1.009408,1200,3,4.5,591
2,6300002,Heat - Edsa Shangri-La,121.056831,14.581404,"Seafood, Asian, Filipino, Indian",0.173592,2.424069,1.143919,0.263210,4000,4,4.4,270
3,6318506,Ooma,121.056475,14.585318,"Japanese, Sushi",0.018581,2.424069,1.473624,0.484048,1500,4,4.9,365
4,6314302,Sambo Kojin,121.057508,14.584450,"Japanese, Korean",0.018581,2.424069,1.407683,0.167901,1500,4,4.8,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9546,5915730,Naml۱ Gurme,28.977392,41.022793,Turkish,-0.069465,1.319726,0.946096,1.467355,80,3,4.1,788
9547,5908749,Ceviz A��ac۱,29.041297,41.009847,"World Cuisine, Patisserie, Cafe",-0.067915,1.319726,1.012037,2.039208,105,3,4.2,1034
9548,5915807,Huqqa,29.034640,41.055817,"Italian, World Cuisine",-0.063885,2.424069,0.682332,1.172130,170,4,3.7,661
9549,5916112,A���k Kahve,29.036019,41.057979,Restaurant Cafe,-0.066985,2.424069,0.880155,1.730035,120,4,4.0,901


In [99]:
#Dealing with multiple values in cuisines
res_data['Cuisines'] = res_data['Cuisines'].str.split(', ')
res_data=res_data.explode('Cuisines')
res_data.head()

,Restaurant ID,Restaurant Name,Longitude,Latitude,Cuisines,Average Cost for two,Price range,Aggregate rating,Votes,avg_cost_for_two,price_range,aggregate_rating,votes
0,6317637,Le Petit Souffle,121.027535,14.565443,French,-0.006221,1.319726,1.407683,0.365493,1100,3,4.8,314
0,6317637,Le Petit Souffle,121.027535,14.565443,Japanese,-0.006221,1.319726,1.407683,0.365493,1100,3,4.8,314
0,6317637,Le Petit Souffle,121.027535,14.565443,Desserts,-0.006221,1.319726,1.407683,0.365493,1100,3,4.8,314
1,6304287,Izakaya Kikufuji,121.014101,14.553708,Japanese,-0.000020,1.319726,1.209860,1.009408,1200,3,4.5,591
2,6300002,Heat - Edsa Shangri-La,121.056831,14.581404,Seafood,0.173592,2.424069,1.143919,0.263210,4000,4,4.4,270


In [100]:
# Extract features from cuisines
vectorizer = TfidfVectorizer()
cuisines_features = vectorizer.fit_transform(res_data['Cuisines'])
cuisines_features

<19710x150 sparse matrix of type '<class 'numpy.float64'>'
	with 27674 stored elements in Compressed Sparse Row format>

In [101]:
# Calculate distance feature
def calculate_distance(lat1, lon1, lat2, lon2):
    coords_1 = (lat1, lon1)
    coords_2 = (lat2, lon2)
    distance = geodesic(coords_1, coords_2).km
    return distance

res_data['Distance'] = res_data.apply(lambda row: calculate_distance(row['Latitude'], row['Longitude'], 0, 0), axis=1)
res_data['dist']=res_data['Distance'].copy()
res_data

,Restaurant ID,Restaurant Name,Longitude,Latitude,Cuisines,Average Cost for two,Price range,Aggregate rating,Votes,avg_cost_for_two,price_range,aggregate_rating,votes,Distance,dist
0,6317637,Le Petit Souffle,121.027535,14.565443,French,-0.006221,1.319726,1.407683,0.365493,1100,3,4.8,314,13349.406667,13349.406667
0,6317637,Le Petit Souffle,121.027535,14.565443,Japanese,-0.006221,1.319726,1.407683,0.365493,1100,3,4.8,314,13349.406667,13349.406667
0,6317637,Le Petit Souffle,121.027535,14.565443,Desserts,-0.006221,1.319726,1.407683,0.365493,1100,3,4.8,314,13349.406667,13349.406667
1,6304287,Izakaya Kikufuji,121.014101,14.553708,Japanese,-0.000020,1.319726,1.209860,1.009408,1200,3,4.5,591,13348.171952,13348.171952
2,6300002,Heat - Edsa Shangri-La,121.056831,14.581404,Seafood,0.173592,2.424069,1.143919,0.263210,4000,4,4.4,270,13352.259620,13352.259620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9547,5908749,Ceviz A��ac۱,29.041297,41.009847,Cafe,-0.067915,1.319726,1.012037,2.039208,105,3,4.2,1034,5404.606058,5404.606058
9548,5915807,Huqqa,29.034640,41.055817,Italian,-0.063885,2.424069,0.682332,1.172130,170,4,3.7,661,5408.133191,5408.133191
9548,5915807,Huqqa,29.034640,41.055817,World Cuisine,-0.063885,2.424069,0.682332,1.172130,170,4,3.7,661,5408.133191,5408.133191
9549,5916112,A���k Kahve,29.036019,41.057979,Restaurant Cafe,-0.066985,2.424069,0.880155,1.730035,120,4,4.0,901,5408.391214,5408.391214


**3) Building Recommendations System**

In [102]:
def transform_user_profile(user_profile, scaler, vectorizer):
    # Transform user profile cuisines to TF-IDF feature space
    user_cuisines = vectorizer.transform([user_profile['Cuisines']])

    # Normalize user profile numerical features
    user_profile_normalized = user_profile.copy()
    user_profile_normalized['Price range'] = scaler.transform([[user_profile_normalized['Price range'], 0, 0, 0]])[0][0]
    user_profile_normalized['Distance'] = scaler.transform([[0, 0, 0, user_profile_normalized['Distance']]])[0][3]

    return user_profile_normalized, user_cuisines

In [103]:
def calculate_combined_similarities(user_profile_normalized, user_cuisines, res_data, cuisines_features):
    # Calculate cosine similarity for cuisines
    cuisines_similarity = cosine_similarity(cuisines_features, user_cuisines).flatten()

    # Calculate numerical features similarity using Euclidean distance
    numerical_features_data = res_data[['Price range', 'Distance']]
    user_numerical_features = np.array([[user_profile_normalized['Price range'], user_profile_normalized['Distance']]])
    numerical_distance = np.apply_along_axis(lambda x: euclidean(x, user_numerical_features[0]), 1, numerical_features_data.values)
    numerical_similarity = 1 / (1 + numerical_distance)  # Convert distance to similarity

    return cuisines_similarity, numerical_similarity

In [104]:
def generate_recommendations(cuisines_similarity, numerical_similarity, res_data, top_n=10):
    combined_similarity = (cuisines_similarity + numerical_similarity) / 2
    res_data['Similarity'] = combined_similarity

    res_data_sorted = res_data.sort_values(by='Similarity', ascending=False)

    top_recommendations = res_data_sorted.head(top_n)
    return top_recommendations

**4) Testing**

In [105]:
#declaring a sample user profile
user_profile = {
    'Price range': 4,  # example value
    'Cuisines': 'French, Japanese, North Indian',  # example value
    'Distance': 13350  # example value in kilometers
}

In [106]:
#transforming user profile
user_profile_normalized, user_cuisines = transform_user_profile(user_profile, scaler, vectorizer)

In [107]:
# Calculating similarities
cuisines_similarity, numerical_similarity = calculate_combined_similarities(user_profile_normalized, user_cuisines, res_data, cuisines_features)

In [108]:
# Generating recommendations
top_recommendations = generate_recommendations(cuisines_similarity, numerical_similarity, res_data, top_n=10)

In [109]:
top_recommendations[['Restaurant Name', 'Cuisines', 'avg_cost_for_two', 'aggregate_rating','votes','dist','Similarity']]

,Restaurant Name,Cuisines,avg_cost_for_two,aggregate_rating,votes,dist,Similarity
1145,La Plage,French,800,4.6,302,0.000000,0.382019
4812,Chez Jerome - Q Cafe,French,1500,0.0,1,0.000000,0.382019
4993,Tokyo Mon Amour,French,2200,3.1,10,0.000000,0.381984
9484,Restaurant Mosaic @ The Orient,French,3210,4.9,85,4143.594060,0.366368
22,Chez Michou,French,55,3.0,6,5540.717148,0.366337
70,Paris 6 Classique,French,200,3.4,73,5672.975790,0.366335
67,Les 3 Brasseurs,French,120,4.6,30,5674.641251,0.366335
9384,Restaurant Gordon Ramsay,French,230,4.7,320,5706.120447,0.366334
9416,The French by Simon Rogan - The Midland,French,160,4.3,114,5931.368823,0.366331
9361,The Kitchin,French,90,4.4,275,6212.634581,0.366327
